In [1]:
import pandas as pd

filtered_df = pd.read_csv('./data/filtered_case_reports.csv')

c:\Users\Filip\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
abstracts = filtered_df['abstract'].tolist()
len(abstracts)

542

In [3]:
# read csv with text queries
queries_df = pd.read_csv('./data/generated_medical_reports.csv')
#add id row to df
ids = [192, 111, 157, 726, 824, 287, 859, 322, 38, 85]
queries_df['id'] = ids
queries_df

,link,Texto Clinico,Observações,id
0,https://casosclinicosonline.spmi.pt/artigos_co...,"S/\nIdentificação: género feminino, 70 anos de...",NaN,192
1,https://casosclinicosonline.spmi.pt/artigos_co...,"S/\nIdentificação: Género feminino, 31 anos\n\...",NaN,111
2,https://casosclinicosonline.spmi.pt/artigos_co...,"S/ \nIdentificação: género feminino, 49 anos,;...",NaN,157
3,https://casosclinicosonline.spmi.pt/artigos_co...,"S/ \nIdentificação: género masculino, 48 anos,...",NaN,726
4,https://casereports.spmi.pt/index.php/cr/artic...,"S/\nIdentidicação: género feminino, 24 anos.\n...",NaN,824
5,https://casosclinicosonline.spmi.pt/artigos_co...,"S/\nIdentificação: Doente do sexo masculino, 7...",NaN,287
6,https://casereports.spmi.pt/index.php/cr/artic...,"S/\nIdentificação: género masculino, 80 anos.\...",NaN,859
7,https://casosclinicosonline.spmi.pt/artigos_co...,"S/\nIdentificação: género feminino, 87 anos, c...",NaN,322
8,https://casosclinicosonline.spmi.pt/artigos_co...,S/\nIdentificação: Doente do género feminino f...,NaN,38
9,https://casosclinicosonline.spmi.pt/artigos_co...,"S/\nIdentificação: género masculino, 51 anos, ...",NaN,85


In [4]:
#test the model on inference given a an abstract and the list of abstracts
from sentence_transformers import SentenceTransformer, util
import torch
# Load the model
#model = SentenceTransformer("models/sentence_transformers/paraphrase-multilingual-mpnet-base-v2/best_model")  #models/sentence_transformers/bert-base-portuguese/best_mode  #"models/sentence_transformers/distiluse-base-multilingual/final_balanced" #models/sentence_transformers/paraphrase-multilingual-mpnet-base-v2/best_model
model = SentenceTransformer("lfcc/medlink-bi-encoder")  #models/sentence_transformers/bert-base-portuguese/best_mode  #"models/sentence_transformers/distiluse-base-multilingual/final_balanced" #models/sentence_transformers/paraphrase-multilingual-mpnet-base-v2/best_model


c:\Users\Filip\AppData\Local\Programs\Python\Python39\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Filip\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
#encode the abstracts
embeddings = model.encode(abstracts, convert_to_tensor=True)

c:\Users\Filip\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [ ]:
retrieved_docs_df = pd.DataFrame(columns=["report_id", "order","abstract_id", "abstract"])
for index, row in queries_df.iterrows():
    query_text = row["Texto Clinico"]
    query_id = row["id"]
    query_embedding = model.encode(query_text, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(query_embedding, embeddings)
    top_results = torch.topk(cosine_scores, k=10)
    rows = []
    
    for score, idx in zip(top_results.values[0], top_results.indices[0]):
        id = idx.cpu().numpy()
        rows.append({"report_id": query_id, "order": 0, "abstract_id": id, "abstract": abstracts[id]})
    retrieved_docs_df = pd.concat([retrieved_docs_df, pd.DataFrame(rows)], ignore_index=True)
        
retrieved_docs_df


,report_id,order,abstract_id,abstract
0,192,0,443,A Síndrome de Cushing (SC) é uma síndrome rara...
1,192,0,50,Introdução: A lesão hepática idiossincrática i...
2,192,0,449,A síndrome de Schnitzler (SS) é uma doença rar...
3,192,0,291,A síndrome de secreção inapropriada de hormona...
4,192,0,319,A Artrite reumatoide (AR) relaciona-se com aum...
...,...,...,...,...
95,85,0,536,Resumo A hipercalcemia secundária ao hiperpara...
96,85,0,300,A Doença de Still do Adulto (DSA) é uma doença...
97,85,0,154,A poliarterite nodosa é uma vasculite necrotiz...
98,85,0,351,A abordagem da febre é sem dúvida uma das arte...


### Corss Encoder Reranking

In [ ]:
from sentence_transformers import CrossEncoder
cross_model = CrossEncoder("lfcc/medlink-cross-encoder", num_labels=1, max_length=512) #bert-case-portuguese-cross-encoder     ms-marco-MiniLM-L-12-v2


In [12]:
# rank the docs fro the retrieved_docs_df dataframe

run_dict = {}
for index, row in queries_df.iterrows():
    query_text = row["Texto Clinico"]
    query_id = row["id"]
    retrieved_docs = retrieved_docs_df[retrieved_docs_df["report_id"] == query_id]
    ranks = cross_model.rank(query_text, retrieved_docs["abstract"].tolist())
    run_dict[f"q_{query_id}"] = {}
    for rank in ranks:
        abstract_idx = rank['corpus_id']  # Get the index of the ranked abstract
        doc_id = retrieved_docs.iloc[abstract_idx]["abstract_id"]
        score = rank['score']
        run_dict[f"q_{query_id}"][f"d_{doc_id}"] =  score

In [13]:
run_dict

{'q_192': {'d_443': 0.5760234,
  'd_50': 0.34181383,
  'd_165': 0.18721057,
  'd_329': 0.16108072,
  'd_131': 0.15967818,
  'd_19': 0.11293766,
  'd_449': 0.041120023,
  'd_43': 0.030603856,
  'd_291': 0.02443663,
  'd_319': 0.023369193},
 'q_111': {'d_89': 0.68024284,
  'd_508': 0.6752792,
  'd_528': 0.67241985,
  'd_56': 0.6620897,
  'd_149': 0.6516882,
  'd_523': 0.64537245,
  'd_327': 0.6051173,
  'd_241': 0.4145496,
  'd_196': 0.3963338,
  'd_413': 0.02947604},
 'q_157': {'d_116': 0.66828716,
  'd_131': 0.5618457,
  'd_466': 0.37680802,
  'd_440': 0.14603738,
  'd_536': 0.1339825,
  'd_201': 0.076329924,
  'd_140': 0.0417772,
  'd_348': 0.031113243,
  'd_478': 0.015763352,
  'd_251': 0.014768788},
 'q_726': {'d_251': 0.70946187,
  'd_329': 0.69459236,
  'd_478': 0.6692833,
  'd_470': 0.6626608,
  'd_417': 0.642294,
  'd_291': 0.40868106,
  'd_533': 0.026743313,
  'd_389': 0.022693284,
  'd_252': 0.018227195,
  'd_463': 0.016373033},
 'q_824': {'d_438': 0.73180133,
  'd_525': 0.727

### Evaluate ranking

In [14]:
#read reranked csv file
import pandas as pd
reranked_df = pd.read_csv("./data/manual_ranked_case_reports.csv")
reranked_df

,report_id,order,abstract_id,abstract,Observações
0,192,0,443,A Síndrome de Cushing (SC) é uma síndrome rara...,NaN
1,192,0,50,Introdução: A lesão hepática idiossincrática i...,NaN
2,192,0,449,A síndrome de Schnitzler (SS) é uma doença rar...,NaN
3,192,0,291,A síndrome de secreção inapropriada de hormona...,NaN
4,192,0,319,A Artrite reumatoide (AR) relaciona-se com aum...,NaN
...,...,...,...,...,...
95,85,0,536,Resumo A hipercalcemia secundária ao hiperpara...,NaN
96,85,0,300,A Doença de Still do Adulto (DSA) é uma doença...,NaN
97,85,3,154,A poliarterite nodosa é uma vasculite necrotiz...,"mialgias, fraqueza muscular proximal"
98,85,0,351,A abordagem da febre é sem dúvida uma das arte...,NaN


In [25]:
reranked_df["order"].value_counts()

order
0     60
1     10
2      9
3      8
5      3
4      3
6      2
7      2
9      1
8      1
10     1
Name: count, dtype: int64

In [16]:
qrels_dict_ordered = {}

for index, row in reranked_df.iterrows():
    query_id = row["report_id"]
    doc_id = row["abstract_id"]
    order = row["order"]
    if f"q_{query_id}" not in qrels_dict_ordered:
        qrels_dict_ordered[f"q_{query_id}"] = {}
    if order != 0:
        qrels_dict_ordered[f"q_{query_id}"][f"d_{doc_id}"] = order
qrels_dict_ordered

{'q_192': {'d_329': 2, 'd_19': 1, 'd_43': 3},
 'q_111': {'d_196': 5,
  'd_327': 4,
  'd_508': 3,
  'd_523': 6,
  'd_528': 9,
  'd_149': 8,
  'd_56': 2,
  'd_89': 1,
  'd_241': 7,
  'd_413': 10},
 'q_157': {'d_131': 2, 'd_116': 1, 'd_348': 3},
 'q_726': {'d_470': 5,
  'd_291': 2,
  'd_478': 1,
  'd_533': 3,
  'd_417': 7,
  'd_251': 4,
  'd_329': 6},
 'q_824': {'d_523': 2, 'd_522': 1, 'd_438': 4, 'd_525': 3, 'd_497': 5},
 'q_287': {'d_1': 1},
 'q_859': {'d_147': 2, 'd_15': 1, 'd_59': 3},
 'q_322': {'d_439': 3, 'd_387': 1, 'd_94': 2},
 'q_38': {'d_61': 2, 'd_434': 1},
 'q_85': {'d_504': 1, 'd_4': 2, 'd_154': 3}}

In [17]:
def convert_order_to_scores(qrels_ordered):
    qrels = {}
    for query, docs in qrels_ordered.items():
        qrels[query] = {doc: 9 - i for i, doc in enumerate(docs)}
    return qrels



def normalize_scores(qrels):
    normalized_qrels = {}
    for query, docs in qrels.items():
        num_docs = len(docs)
        # Normalize scores between 0 and 1 based on their order
        normalized_qrels[query] = {
            doc: (num_docs - rank + 1) / num_docs  # Higher rank gets a higher score
            for rank, (doc, _) in enumerate(sorted(docs.items(), key=lambda item: item[1]), start=1)
        }
    return normalized_qrels



qrels_dict = convert_order_to_scores(qrels_dict_ordered)
#qrels_dict = normalize_scores(qrels_dict_ordered)
print(qrels_dict)

{'q_192': {'d_329': 9, 'd_19': 8, 'd_43': 7}, 'q_111': {'d_196': 9, 'd_327': 8, 'd_508': 7, 'd_523': 6, 'd_528': 5, 'd_149': 4, 'd_56': 3, 'd_89': 2, 'd_241': 1, 'd_413': 0}, 'q_157': {'d_131': 9, 'd_116': 8, 'd_348': 7}, 'q_726': {'d_470': 9, 'd_291': 8, 'd_478': 7, 'd_533': 6, 'd_417': 5, 'd_251': 4, 'd_329': 3}, 'q_824': {'d_523': 9, 'd_522': 8, 'd_438': 7, 'd_525': 6, 'd_497': 5}, 'q_287': {'d_1': 9}, 'q_859': {'d_147': 9, 'd_15': 8, 'd_59': 7}, 'q_322': {'d_439': 9, 'd_387': 8, 'd_94': 7}, 'q_38': {'d_61': 9, 'd_434': 8}, 'q_85': {'d_504': 9, 'd_4': 8, 'd_154': 7}}


In [18]:
from ranx import Qrels, Run, evaluate, compare
qrels = Qrels(qrels_dict, name="cross_encoder")
run = Run(run_dict,  name="cross_encoder")

In [23]:
metrics_list = ["mrr@1", "map@1", "ndcg@1", "hit_rate@1", "precision@1", "recall@1", "mrr@3", "map@3", "ndcg@3", "hit_rate@3", "precision@3", "recall@3", "mrr@5", "map@5", "ndcg@5","hit_rate@5", "precision@5", "recall@5", "mrr@10", "map@10", "ndcg@10","hit_rate@10", "precision@10", "recall@10"]
evaluate(qrels, run, metrics_list)

{'mrr@1': 0.6,
 'map@1': 0.14539682539682539,
 'ndcg@1': 0.4333333333333334,
 'hit_rate@1': 0.6,
 'precision@1': 0.6,
 'recall@1': 0.14539682539682539,
 'mrr@3': 0.6833333333333333,
 'map@3': 0.4056349206349206,
 'ndcg@3': 0.5168620572217493,
 'hit_rate@3': 0.8,
 'precision@3': 0.6,
 'recall@3': 0.46952380952380957,
 'mrr@5': 0.7083333333333333,
 'map@5': 0.5680952380952381,
 'ndcg@5': 0.6202352541390936,
 'hit_rate@5': 0.9,
 'precision@5': 0.5599999999999999,
 'recall@5': 0.6936507936507936,
 'mrr@10': 0.7249999999999999,
 'map@10': 0.7081746031746032,
 'ndcg@10': 0.747301756143202,
 'hit_rate@10': 1.0,
 'precision@10': 0.38999999999999996,
 'recall@10': 1.0}

In [20]:
report = compare(
    qrels=qrels,
    runs=[run],
    metrics=metrics_list,
    max_p=0.01  # P-value threshold
)
print(report)

#    Model            MRR@1    MAP@1    NDCG@1    Hit Rate@1    P@1    Recall@1    MRR@3    MAP@3    NDCG@3    Hit Rate@3    P@3    Recall@3    MRR@5    MAP@5    NDCG@5    Hit Rate@5    P@5    Recall@5    MRR@10    MAP@10    NDCG@10    Hit Rate@10    P@10    Recall@10
---  -------------  -------  -------  --------  ------------  -----  ----------  -------  -------  --------  ------------  -----  ----------  -------  -------  --------  ------------  -----  ----------  --------  --------  ---------  -------------  ------  -----------
a    cross_encoder      0.6    0.145     0.433           0.6    0.6       0.145    0.683    0.406     0.517           0.8    0.6        0.47    0.708    0.568      0.62           0.9   0.56       0.694     0.725     0.708      0.747              1    0.39            1
